In [1]:
!pip install --upgrade azureml-sdk[databricks]
!pip install xlrd

In [2]:
from azureml.core.dataset import Dataset
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions, Row
from decimal import Decimal
from datetime import datetime
from pyspark import SparkFiles
import pandas as pd
import xlrd
from pyspark.sql import SQLContext
import os
from azureml.core import Workspace, Datastore
from azureml.core.authentication import AzureCliAuthentication
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import uuid
from datetime import date
from sklearn.model_selection import train_test_split

In [3]:
dbutils.fs.mount(
  source = "wasbs://amldemocontainer@amldemosa.blob.core.windows.net",
  mount_point = "/mnt/finaldata",
  extra_configs = {"fs.azure.account.key.amldemosa.blob.core.windows.net":dbutils.secrets.get(scope = "secret-scope", key = "amldemosakey")})

Out[82]: True

In [4]:
appid = "f7a784e8-9194-4816-872f-d1439502da4f"
appkey = "9804J-7ko39Wz9PX2_ak~Pw-4wis6tnEDK"
tenantid = "72f988bf-86f1-41af-91ab-2d7cd011db47"

container = "amldemocontainer"
storageAccountName = "amldemosa.dfs.core.windows.net"

In [5]:
new = (spark.read.format("com.microsoft.cdm")
  .option("storage", storageAccountName)
  .option("container", container)
  .option("manifest", "/prepared-by-data-bricks/default.manifest.cdm.json")
  .option("entity", "airbnb")
  .option("appId", appid)
  .option("appKey", appkey)
  .option("tenantId", tenantid)
  .load())
keepcolumns = ["host_id","room_type", "neighborhood", "reviews", "overall_satisfaction", "accommodates", "bedrooms", "price"]
airbnb = new[keepcolumns]
display(airbnb)

room_id,survey_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,name,last_modified,latitude,longitude,ArrivalTime
17237031,1476,12796071,Entire home/apt,Oostelijk Havengebied / Indische Buurt,9,4.5,2,1,119.0,House in Amsterdam East.,2017-07-22 17:51:44.053411,52.36461,4.9332129999999985,2020-06-16T14:42:01.825+0000
13160246,1476,2969181,Entire home/apt,De Baarsjes / Oud West,5,5.0,4,2,179.0,Lovely 4p apartment in the vibrant West,2017-07-22 22:58:02.919951,52.373838,4.861415,2020-06-16T14:42:06.888+0000
237017,1476,1158389,Entire home/apt,De Baarsjes / Oud West,14,5.0,2,1,96.0,Cozy & stylish near Jordan/Centre,2017-07-22 17:31:12.466448,52.371493,4.862764,2020-06-16T14:42:11.952+0000
1512777,1476,4272620,Entire home/apt,De Baarsjes / Oud West,19,5.0,4,2,300.0,Lovely big house in centre + garden,2017-07-23 03:42:56.742346,52.36103,4.861597,2020-06-16T14:42:17.000+0000
13135723,1476,48304746,Entire home/apt,Ijburg / Eiland Zeeburg,0,0.0,4,2,210.0,Modern villa in Amsterdam IJburg,2017-07-23 03:00:07.378574,52.359133,4.981972,2020-06-16T14:42:22.079+0000
18694086,1476,13254106,Entire home/apt,De Baarsjes / Oud West,1,0.0,4,2,144.0,Appartement located close to centre and Vondelpark,2017-07-22 20:28:35.182770,52.361708,4.86326,2020-06-16T14:42:27.142+0000
9955618,1476,3256461,Entire home/apt,De Baarsjes / Oud West,1,0.0,4,2,378.0,135 m2 newly renovated apartment,2017-07-23 05:55:32.679292,52.368294,4.853459,2020-06-16T14:35:01.732+0000
19915942,1476,141071362,Entire home/apt,Oostelijk Havengebied / Indische Buurt,0,0.0,2,1,174.0,"Spacious, Amazing View! @Hip Area, close to Center",2017-07-22 22:16:10.646653,52.365086,4.933634,2020-06-16T14:35:06.841+0000
9673184,1476,49982971,Entire home/apt,De Baarsjes / Oud West,11,4.5,2,1,179.0,Perfect apartment in the city,2017-07-22 22:31:59.609532,52.364085,4.8771,2020-06-16T14:35:11.904+0000
18154862,1476,95736049,Entire home/apt,Ijburg / Eiland Zeeburg,3,5.0,6,4,216.0,Wonderfull Family House at Lake and Park,2017-07-23 03:00:05.682878,52.35591,4.986856,2020-06-16T14:35:16.966+0000


In [6]:
reference = (spark.read.format("com.microsoft.cdm")
  .option("storage", storageAccountName)
  .option("container", container)
  .option("manifest", "/prepared-by-dataflows/default.manifest.cdm.json")
  .option("entity", "Reference")
  .option("appId", appid)
  .option("appKey", appkey)
  .option("tenantId", tenantid)
  .load())

display(reference)

quality,neighborhood,distance_from_airport
excellent,Centrum West,8
excellent,De Pijp / Rivierenbuurt,5
good,De Baarsjes / Oud West,6
very bad,Watergraafsmeer,4
very good,Westerpark,7
bad,Oostelijk Havengebied / Indische Buurt,9
very good,Centrum Oost,8
good,Oud Oost,8
good,Buitenveldert / Zuidas,3
terrible,Geuzenveld / Slotermeer,4


In [7]:
data = airbnb.join(reference, "neighborhood", how='left_outer')
display(data)

neighborhood,host_id,room_type,reviews,overall_satisfaction,accommodates,bedrooms,price,quality,distance_from_airport
De Aker / Nieuw Sloten,13960296,Entire home/apt,0,0.0,6,4,330.0,very bad,5
Geuzenveld / Slotermeer,91491112,Entire home/apt,5,4.5,2,1,119.0,terrible,4
Geuzenveld / Slotermeer,1108566,Private room,61,5.0,2,1,66.0,terrible,4
Bos en Lommer,45425600,Private room,20,5.0,2,1,75.0,good,5
Bos en Lommer,14460671,Entire home/apt,23,5.0,4,3,166.0,good,5
Bos en Lommer,9589947,Entire home/apt,0,0.0,4,2,365.0,good,5
Bos en Lommer,19109502,Private room,14,5.0,4,1,89.0,good,5
Bos en Lommer,61966560,Entire home/apt,0,0.0,2,1,119.0,good,5
Bos en Lommer,5403147,Entire home/apt,2,0.0,2,0,66.0,good,5
Bos en Lommer,58428526,Entire home/apt,5,4.5,4,1,180.0,good,5


In [8]:
url = "http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2020-06-08/visualisations/listings.csv"
df = pd.read_csv(url)
test = (sqlContext.createDataFrame(df))
keepcolumn = ["host_id","host_name","minimum_nights", "calculated_host_listings_count","availability_365"]
extra =  test[keepcolumn]
display(extra)

host_id,host_name,minimum_nights,calculated_host_listings_count,availability_365
3159,Daniel,3,1,158
32366,Sabine And Sander,3,1,17
59484,Alexander,1,2,31
56142,Joan,14,1,212
97647,Flip,2,1,235
124245,Edwin,2,3,258
124245,Edwin,2,3,277
133488,Nienke,3,1,159
178515,Fatih,4,1,16
187580,Micha,3,2,321


In [9]:
final = data.join(extra, "host_id", how='left_outer')
final.na.drop(subset=["host_id"])
display(final)

host_id,neighborhood,room_type,reviews,overall_satisfaction,accommodates,bedrooms,price,quality,distance_from_airport,host_name,minimum_nights,calculated_host_listings_count,availability_365
15225846,De Baarsjes / Oud West,Entire home/apt,2,0.0,4,3,232.0,good,6,Family,3,1,0
13689090,De Pijp / Rivierenbuurt,Entire home/apt,16,4.5,4,1,114.0,excellent,5,Nicole,3,1,0
31675383,Noord-West / Noord-Midden,Entire home/apt,0,0.0,2,1,147.0,bad,11,Floris,2,1,0
141272694,Noord-West / Noord-Midden,Entire home/apt,0,0.0,4,2,172.0,bad,11,Matthijs,2,1,0
10971362,Slotervaart,Private room,11,5.0,2,1,84.0,very bad,4,null,null,null,null
48703385,De Baarsjes / Oud West,Entire home/apt,1,0.0,4,3,270.0,good,6,Rowan,1,2,0
48703385,De Baarsjes / Oud West,Entire home/apt,1,0.0,4,3,270.0,good,6,Rowan,2,2,47
52390138,Centrum West,Entire home/apt,13,4.5,2,1,132.0,excellent,8,null,null,null,null
312454,Centrum Oost,Entire home/apt,48,4.5,2,1,204.0,very good,8,Rachelle,1,1,0
17906944,Westerpark,Entire home/apt,12,5.0,2,1,180.0,very good,7,null,null,null,null


In [10]:
final.write.parquet('/mnt/finaldata/demo/finaldata')

In [11]:
(extra.write.format("com.microsoft.cdm")
  .option("storage", storageAccountName)
  .option("container", container)
  .option("manifest", "/prepared-by-data-bricks/default.manifest.cdm.json")
  .option("entity", "hostinfo")
  .option("format", "parquet")
  .option("appId", appid)
  .option("appKey", appkey)
  .option("tenantId", tenantid)
  .save())

In [12]:
datastore_name = 'featurestore'

# get existing ML workspace
workspace = Workspace(
  subscription_id="e0eeddf8-2d02-4a01-9786-92bb0e0cb692", 
  resource_group="AMLdemo", 
  workspace_name="amldemoaml"
)

# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(workspace, datastore_name)

In [13]:
dataset = Dataset.Tabular.from_delimited_files(
            path="https://amldemosa.blob.core.windows.net/amldemocontainer/demo/finaldata/*.parquet"
        )

In [14]:
dataset.register(
            workspace=workspace,
            name=("datafinal"),
            description='final airbnbdata',
            create_new_version=True
        )

Out[104]: {
 "source": [
 "https://amldemosa.blob.core.windows.net/amldemocontainer/demo/finaldata/*.parquet"
 ],
 "definition": [
 "GetFiles",
 "ParseDelimited",
 "DropColumns",
 "SetColumnTypes"
 ],
 "registration": {
 "id": "cdb98f04-df6e-458f-aa8c-5bceea81de11",
 "name": "datafinal",
 "version": 3,
 "description": "final airbnbdata",
 "workspace": "Workspace.create(name='amldemoaml', subscription_id='e0eeddf8-2d02-4a01-9786-92bb0e0cb692', resource_group='AMLdemo')"
 }
}

In [15]:
!az login